<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#scrap-championships" data-toc-modified-id="scrap-championships-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>scrap championships</a></span></li><li><span><a href="#scrap-teams-and-players" data-toc-modified-id="scrap-teams-and-players-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>scrap teams and players</a></span></li><li><span><a href="#scrap-matches" data-toc-modified-id="scrap-matches-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>scrap matches</a></span></li><li><span><a href="#add-meteo-to-the-df_matches" data-toc-modified-id="add-meteo-to-the-df_matches-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>add meteo to the df_matches</a></span></li><li><span><a href="#scrap-goals-par-player/match" data-toc-modified-id="scrap-goals-par-player/match-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>scrap goals par player/match</a></span></li><li><span><a href="#add-df-to-sql-database" data-toc-modified-id="add-df-to-sql-database-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>add df to sql database</a></span><ul class="toc-item"><li><span><a href="#create-championship-table-sql" data-toc-modified-id="create-championship-table-sql-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>create championship table sql</a></span></li><li><span><a href="#create-teams-table-in-sql" data-toc-modified-id="create-teams-table-in-sql-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>create teams table in sql</a></span></li><li><span><a href="#create-participation-table" data-toc-modified-id="create-participation-table-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>create participation table</a></span></li><li><span><a href="#create-players-table" data-toc-modified-id="create-players-table-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>create players table</a></span></li><li><span><a href="#create-matches-table" data-toc-modified-id="create-matches-table-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>create matches table</a></span></li><li><span><a href="#create-matches_teams-table" data-toc-modified-id="create-matches_teams-table-6.6"><span class="toc-item-num">6.6&nbsp;&nbsp;</span>create matches_teams table</a></span></li><li><span><a href="#create-goals-table" data-toc-modified-id="create-goals-table-6.7"><span class="toc-item-num">6.7&nbsp;&nbsp;</span>create goals table</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd
import matplotlib.pyplot  as plt
import seaborn as sns

In [2]:
url = "https://www.lequipe.fr/Football/"
main_url = 'https://www.lequipe.fr'

In [35]:
# function to get page soup from html page
import requests
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq

def get_page_soup(url):
    # opening connection
    uClient = uReq(url, timeout=15)
    page_html = uClient.read()
    uClient.close()
    #html parser
    return soup(page_html, "html.parser")


### scrap championships

In [4]:
#get championships links
main_soup = get_page_soup(url)
#equipe_soup
#a aria-label = '/Football/' class = 'Link Menu__accordionTitle'
championships = main_soup.find("div", {"class": 'Menu__accordion js-menu-accordion is-active'}).findAll("li", {"class":"Menu__item" })
print(championships[0].a['href'])
championships_dict = {'name':[], 'link':[]}
for champ in championships:
    championships_dict['name'].append(champ.a.text.split('\n')[1].strip())
    championships_dict['link'].append(champ.a['href'])

df_championships = pd.DataFrame(championships_dict)
df_championships.head()

https://www.lequipe.fr/Football/Euro/


,name,link
0,Euro,https://www.lequipe.fr/Football/Euro/
1,Copa America,https://www.lequipe.fr/Football/copa-america/p...
2,Ligue 1,https://www.lequipe.fr/Football/ligue-1/page-c...
3,Ligue 2,https://www.lequipe.fr/Football/ligue-2/page-c...
4,Ligue des Champions,https://www.lequipe.fr/Football/ligue-des-cham...


In [5]:
results_link_list = []; teams_link_list = []; matches_link_list = []

for i in range(len(df_championships)):
    url_champ = df_championships.loc[i,'link']
    # insert code below here after
    
    # for single championship
    #url_champ = df_championships.loc[2,'link']
    page_championship = get_page_soup(url_champ)
    navigator_items = page_championship.find("div", {"class": "MainNav__items js-mainNavItems"}).findAll("a", {"class": "Link MainNav__item"})

    results_link = None; teams_link = None; matches_link = None

    for item in navigator_items:

        if "résultats" in item.text.strip().split():
            #print("res")
            results_link = item["href"]
        if "sélections" in item.text.strip().split() or "clubs" in item.text.strip().split():
            #print('TEAMS link: {}'.format(item["href"]))   
            teams_link = item["href"]
        if "matches" in item.text.strip().split() or "matchs" in item.text.strip().split():
            #print('MATCHES link: {}'.format(item["href"]))
            matches_link = item['href']

    results_link_list.append(results_link)
    teams_link_list.append(teams_link)
    matches_link_list.append(matches_link)

df_championships['result_link'] = results_link_list
df_championships['teams_link'] = teams_link_list
df_championships['matches_link'] = matches_link_list

In [6]:
df_championships.isnull().sum()

name             0
link             0
result_link     43
teams_link      14
matches_link     9
dtype: int64

### scrap teams and players



In [7]:
## create database teams

In [8]:
# for i in range(len(df_championships)):
#     url_teams = df_championships.loc[i,'teams_link']

url_teams = df_championships.loc[2,'teams_link']
champ_name = df_championships.loc[2, 'name']
## champ_year_start = df_championships.loc[2, 'year_start']

teams_page = get_page_soup(url_teams)

teams = teams_page.find("ul", {"class": "clubs"}).findAll("li")
team_links = [team.h2.a['href'] for team in teams if team.h2 is not None]
team_links = [main_url + link for link in team_links if 'https://' not in link ]
team_names = [team.h2.a.text.strip() for team in teams if team.h2 is not None]


In [9]:
# get info for each team

dict_teams = {
    'city': [],
    'name': [],
    'coach_name': [],
    'president': [],
    'year_founded': [],
    'championship': []
}
dict_players = {
    'number': [],
    'name': [],
    'country': [],
    'birthday': [],
    'position': [],
    'club': []
}
df_players = pd.DataFrame(dict_players)
#print(df_players.head())
for i, (link, name) in enumerate(zip(team_links, team_names)):

#     link = df_teams_links.loc[i, 'links']
#     name = df_teams_links.loc[i, 'team_short_names']
    #print(name)
    # team info and player info dirty tables
    team_info = pd.read_html(link)[0]

    dict_teams['city'].append(name)
    dict_teams['name'].append(team_info.loc[0, 0].split(':')[1].strip())
    dict_teams['year_founded'].append(team_info.loc[1, 0].split(':')[1].strip())
    dict_teams['president'].append(team_info.loc[2, 0].split(':')[1].strip())
    dict_teams['coach_name'].append(team_info.loc[3, 0].split(':')[1].strip())
    dict_teams['championship'].append(champ_name)

    # players info
    if len(pd.read_html(link)[2]) > 5:
        team_players = pd.read_html(link)[2]
    else:
        team_players = pd.read_html(link)[1]

    old_column_names = team_players.columns
    team_players = team_players.rename(
        columns={
            'Unnamed: 0': 'number',
            'Nom': 'name',
            'Pays': 'country',
            old_column_names[3]: 'birthday',
            'Pos.': 'position'
        })
    team_players['club'] = name

    df_players = pd.concat([df_players, team_players], ignore_index=True)
    #print(df_players.tail(2))

df_teams = pd.DataFrame(dict_teams)




In [10]:
df_players.isnull().sum()

number      106
name          0
country       0
birthday      5
position      0
club          0
dtype: int64

In [51]:
def change_date_format(a):
    if a is not np.nan:
        a = a.split("/")
        a[-1] = str(int(a[-1])+2000) if int(a[-1]) < 30 else str(int(a[-1]) +1900) 
        a.reverse()
        a = '-'.join(a)
    return(a)

df_players['birthday'] = df_players['birthday'].apply(change_date_format)



ValueError: invalid literal for int() with base 10: '1997-04-18'

In [115]:
df_players['name'].unique()

array(['P. Bernardoni', 'L. Butelle', 'D. Petkovic', 'A. Bamba',
       'K. Boma', 'E. Diaw', 'S. Doumbia', 'E. Ebosse', 'V. Manceau',
       'M. Pavlovic', 'R. Thomas', 'I. Traoré', 'I. Amadou', 'K. Bemanga',
       'A. Bobichon', 'S. Boufal', 'J. Cabot', 'P. Capelle', 'M. Cho',
       'L. Coulibaly', 'A. Fulgini', 'T. Mangani', 'Z. Ould Khaled',
       'M. Pereira Lage', 'W. Taïbi', 'R. Alioui', 'S. Bahoken',
       'L. Diony', 'F. El-Melali', 'N. Fatar', 'Y. Fortuné', 'S. Thioub',
       'B. Costil', 'D. Lima Semedo', 'C. Michel', 'G. Poussin',
       'D. Rouyard', 'P. Baysse', 'L. Benito', 'L. Bessilé',
       'M. Bokélé Mputu', 'D. Cardoso', 'V. Jovanovic', 'L. Koscielny',
       'E. Kwateng', 'E. Mexer', 'M. Poundjé', 'I. Sow', 'Y. Adli',
       'T. Basic', 'T. Lacoux', 'Otavio', 'J. Seri', 'I. Sissokho',
       'M. Zerkane', 'D. Bakwa', 'H. Ben Arfa', 'J. Briand',
       'N. De Préville', 'L. Delaurier-Ch...', 'I. Diarra', 'U. Hwang',
       'S. Kalu', 'S. Mara', 'R. Oudin', 'A.

In [12]:
df_teams.tail(3)

,city,name,coach_name,president,year_founded,championship
17,Rennes,Stade Rennais Football Club,Bruno GENESIO,Nicolas Holveck,1901,Ligue 1
18,Saint-Étienne,Association Sportive de St-Etienne,Claude PUEL,B. Caïazzo & R. Romeyer,1919,Ligue 1
19,Strasbourg,Racing Club de Strasbourg Alsace,Julien STEPHAN,Marc Keller,1906,Ligue 1


In [13]:
df_players.head(3)


,number,name,country,birthday,position,club
0,1.0,P. Bernardoni,FRA,1997-04-18,Gar.,Angers
1,16.0,L. Butelle,FRA,1983-04-3,Gar.,Angers
2,30.0,D. Petkovic,MNE,1993-05-25,Gar.,Angers


### scrap matches 

In [14]:
link_matches = 'https://www.lequipe.fr/Football/ligue-1/saison-2020-2021/page-calendrier-resultats'
matches_page = get_page_soup(link_matches)

matches_json = []
matches = matches_page.find("select",{"class":"SelectNav__select"}).findAll('option')
for match in matches:
    matches_json.append(match['value'])


In [22]:
#data from one json table
dict_matches = {
    'date': [],
    'place': [],
    'team_domicile': [],
    'team_exterieur': [],
    'score_domicile': [],
    'score_exterieur': [],
    'vainqueur': []
}


import json
import requests

for match_json in matches_json:
    response = requests.get(match_json)
    data = json.loads(response.text)
    for day in data['items'][:-1]:
        
        for team in day['items']:
            dict_matches["place"].append(team['event']['lieu']['ville'])
            dict_matches["date"].append(team['date'])
            dict_matches["team_domicile"].append(team['event']['specifics']['domicile']['equipe'][
                'nom'])
            dict_matches["team_exterieur"].append(team['event']['specifics']['exterieur']['equipe'][
                'nom'])
            dict_matches["score_domicile"].append(team['event']['specifics']['score']['domicile'])
            dict_matches["score_exterieur"].append(team['event']['specifics']['score']['exterieur'])
            dict_matches["vainqueur"].append(team['event']['specifics']['vainqueur'])

            #print("Stade:{} Date:{}".format(lieu, date))
            #print("{} {}:{} {}".format(team_domicile, score_domicile, score_exterieur, team_exterieur))
            #print("\n")
            
            
df_matches_score = pd.DataFrame(dict_matches)

In [16]:
# from datetime import datetime
# df_matches_score['date']= [ datetime.strptime(date, '%Y-%m-%dT%H:%M:%S%z') for date in df_matches_score['date'] ]

### add meteo to the df_matches

### scrap goals par player/match

In [36]:
#import urllib.request
#from bs4 import BeautifulSoup
import csv
import re
import json
from http.client import IncompleteRead
import numpy as np

url_match_list = []
url = ''
for i in np.arange(1, 39):
    if i == 1:
        url = f'https://www.lequipe.fr/Football/ligue-1/saison-2020-2021/page-calendrier-resultats/{i}re-journee'
    else:
        url = f'https://www.lequipe.fr/Football/ligue-1/saison-2020-2021/page-calendrier-resultats/{i}e-journee'

    page_soup = get_page_soup(url)
    matches_json = []
    main = page_soup.find('div', attrs={'class': 'container__main'})
    teams_html = main.findAll('div', attrs={'class': 'TeamScore__data'})
    for l in teams_html:
        link_teams = l.find('a', attrs={'class': 'Link'}, href=True)
        a = link_teams['href']
        url_match_list.append(a)

In [37]:
home_team_goals = []
exterior_team_goals = []
for url in url_match_list:

    page_soup = get_page_soup(url)
    goal_list = page_soup.findAll(
        'div', attrs={'class': 'grid__content Scoreboard__goalList'})

    goal_list_right = []
    goal_list_left = []

    if goal_list == []:
        home_team_goals.append([])
        exterior_team_goals.append([])
    else:

        for s in goal_list[0].findAll('span',
                                      attrs={'class': 'Scoreboard__scorer'}):

            goal_list_left.append(s.text)

        for s in goal_list[1].findAll('span',
                                      attrs={'class': 'Scoreboard__scorer'}):

            goal_list_right.append(s.text)

        home_team_goals.append(goal_list_left)
        exterior_team_goals.append(goal_list_right)

In [39]:
# create database
exterior_team_goals

[[],
 ['I. Traoré'],
 ['D. Da Silva'],
 ['B. Dia', 'E. Touré'],
 ['M. Chahiri'],
 [],
 ['G. Kakuta'],
 ['M. Depay'],
 [],
 ['R. Hamouma', 'D. Bouanga'],
 ['A. Scheidler'],
 ['G. Laborde'],
 ['K. Dolberg', 'K. Dolberg'],
 ['J. Bamba'],
 ['Z. Ferhat'],
 [],
 ['J. Maja', 'T. Basic'],
 ['B. Badiashile'],
 ['F. Thauvin', 'D. Caleta-Car', 'D. Caleta-Car'],
 [],
 [],
 ['Dante'],
 [],
 [],
 ['S. Guirassy', 'S. Guirassy', 'N. Aguerd', 'B. Bourigeaud'],
 ['G. Kakuta', 'F. Medina', 'I. Ganago'],
 [],
 ['R. Perraud', 'I. Cardona'],
 ['L. Blas'],
 ['F. Thauvin'],
 [],
 ['S. Kalu'],
 ['W. Ben Yedder'],
 ['K. Mbappé', 'A. Di Maria', 'Marquinhos'],
 ['S. Bahoken'],
 ['C. Hérelle', 'T. Monconduit'],
 ['B. Dia'],
 [],
 ['A. Aouchiche', 'Y. Maçon'],
 ['L. Araujo'],
 [],
 ['N. Aguerd', 'S. Guirassy', 'A. Hunou'],
 ['I. Niane'],
 [],
 ['A. Delort', 'T. Savanier'],
 ['I. Ganago'],
 ['M. Chahiri', 'L. Ajorque'],
 ['S. Mounié', 'G. Charbonnier'],
 ['L. Dubois'],
 ['M. Icardi', 'M. Icardi'],
 ['I. Traoré'],
 [

In [45]:
df_matches_score['goals_domicile'] = home_team_goals

In [46]:
df_matches_score['goals_exterieur'] = exterior_team_goals

In [48]:
df_matches_score.tail()

,date,place,team_domicile,team_exterieur,score_domicile,score_exterieur,vainqueur,goals_domicile,goals_exterieur
375,2021-05-23T21:00:00+0200,Rennes,Rennes,Nîmes,2,0,domicile,"[S. Guirassy, B. Bourigeaud]",[]
376,2021-05-23T21:00:00+0200,Nantes,Nantes,Montpellier,1,2,exterieur,[R. Kolo Muani],"[G. Laborde, A. Delort]"
377,2021-05-23T21:00:00+0200,Saint-Étienne,Saint-Étienne,Dijon,0,1,exterieur,[],[A. Kamara]
378,2021-05-23T21:00:00+0200,Reims,Reims,Bordeaux,1,2,exterieur,[E. Touré],"[Y. Adli, E. Kwateng]"
379,2021-05-23T21:00:00+0200,Strasbourg,Strasbourg,Lorient,1,1,ex-aequo,[H. Diallo],[T. Chalobah]


In [59]:
display(df_matches_score.head())
# df_matches
df_matches = df_matches_score[['date','place']]
df_matches.is_copy = None
display(df_matches.head())

# df_matches_teams
df_matches_teams1 = df_matches_score[['date','place','team_domicile','score_domicile', 'goals_domicile']].rename(columns = {'team_domicile': 'team', 'score_domicile': 'team_goal', 'goals_domicile': 'players'})
df_matches_teams2 = df_matches_score[['date','place','team_exterieur','score_exterieur', 'goals_exterieur']].rename(columns = {'team_exterieur': 'team', 'score_exterieur': 'team_goal', 'goals_exterieur': 'players'})
df_matches_teams = pd.concat([df_matches_teams1, df_matches_teams2], ignore_index=True)
df_matches_teams = df_matches_teams.sort_values(by='date').reset_index(drop=True)
display(df_matches_teams.head(10))   

,date,place,team_domicile,team_exterieur,score_domicile,score_exterieur,vainqueur,goals_domicile,goals_exterieur
0,2020-08-21T19:00:00+0200,Bordeaux,Bordeaux,Nantes,0,0,ex-aequo,[],[]
1,2020-08-22T17:00:00+0200,Dijon,Dijon,Angers,0,1,exterieur,[],[I. Traoré]
2,2020-08-22T21:00:00+0200,Lille,Lille,Rennes,1,1,ex-aequo,[J. Bamba],[D. Da Silva]
3,2020-08-23T13:00:00+0200,Monaco,Monaco,Reims,2,2,ex-aequo,"[A. Disasi, B. Badiashile]","[B. Dia, E. Touré]"
4,2020-08-23T15:00:00+0200,Lorient,Lorient,Strasbourg,3,1,domicile,"[Y. Wissa, A. Grbic, P. Hamel]",[M. Chahiri]


,date,place
0,2020-08-21T19:00:00+0200,Bordeaux
1,2020-08-22T17:00:00+0200,Dijon
2,2020-08-22T21:00:00+0200,Lille
3,2020-08-23T13:00:00+0200,Monaco
4,2020-08-23T15:00:00+0200,Lorient


,date,place,team,team_goal,players
0,2020-08-21T19:00:00+0200,Bordeaux,Bordeaux,0,[]
1,2020-08-21T19:00:00+0200,Bordeaux,Nantes,0,[]
2,2020-08-22T17:00:00+0200,Dijon,Dijon,0,[]
3,2020-08-22T17:00:00+0200,Dijon,Angers,1,[I. Traoré]
4,2020-08-22T21:00:00+0200,Lille,Rennes,1,[D. Da Silva]
5,2020-08-22T21:00:00+0200,Lille,Lille,1,[J. Bamba]
6,2020-08-23T13:00:00+0200,Monaco,Monaco,2,"[A. Disasi, B. Badiashile]"
7,2020-08-23T13:00:00+0200,Monaco,Reims,2,"[B. Dia, E. Touré]"
8,2020-08-23T15:00:00+0200,Lorient,Lorient,3,"[Y. Wissa, A. Grbic, P. Hamel]"
9,2020-08-23T15:00:00+0200,Nîmes,Brest,0,[]


In [61]:
df_matches['temperature'] = [18]*len(df_matches)
df_matches['rainfall'] = [0]*len(df_matches)
df_matches.tail()
print(len(df_matches_score))

380


<ipython-input-61-8d71e27cafc8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matches['temperature'] = [18]*len(df_matches)
<ipython-input-61-8d71e27cafc8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matches['rainfall'] = [0]*len(df_matches)


### add df to sql database 

In [62]:
import sqlite3
class ChampionshipsDataBase:


    def __init__(self, name='championships.sqlite'):
        self.name_db = name

        if os.path.isfile(
                self.name_db):  # if sqlite file exist create a connection
            self.connection = self.__connect_db()
            self.cur = self.connection.cursor()
            print(self.connection)

        else:  # if file sqlite doesn't exist create file and then create sql

            open(self.name_db, 'x')
            self.connection = self.__connect_db()
            print(self.connection)
            self.cur = self.connection.cursor()
            self.__create_db()
            #self.fill_db()

        

    def get_tables_names(self):
        self.cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

        #store all names of the tables/chemas in var tables
        tables0 = self.cur.fetchall()

        tables = []
        for tab in tables0:
            tables.append(tab[0])

        return tables
    

    def get_df(self, table_name):
        return pd.read_sql_query(f"SELECT * FROM {table_name}",
                                 self.connection)

    def __connect_db(self):
        return sqlite3.connect(self.name_db)
    
    def __create_db(self):
        
        conn = self.connection
        cursor = self.cur

        # cursor.execute('DROP TABLE championships;')
        # cursor.execute('DROP TABLE matches;')
        # cursor.execute('DROP TABLE teams;')
        # cursor.execute('DROP TABLE matches_teams;')
        # cursor.execute('DROP TABLE players;')
        # cursor.execute('DROP TABLE goals;')

        #championships
        cursor.execute("""
        CREATE TABLE championships(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name VARCHAR(255) NOT NULL UNIQUE,
        country VARCHAR(255),
        start_year INTEGER,
        end_year INTEGER
        )
        ;""")

        conn.commit()
           
        #teams
        cursor.execute("""
        CREATE TABLE teams(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name VARCHAR(255) NOT NULL UNIQUE,
        city VARCHAR(255),
        coach_name VARCHAR(255),
        year_founded INTEGER
        )
        ;""")
        conn.commit()
               
        #participation
        cursor.execute("""
        CREATE TABLE participation(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        championship_id INTEGER,
        team_id INTEGER,
        FOREIGN KEY (championship_id)
        REFERENCES championships(id),
        FOREIGN KEY (team_id)
        REFERENCES teams(id)
        )
        ;""")
        conn.commit()

        
        #matches
        cursor.execute("""
        CREATE TABLE matches(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date DATE,
        place VARCHAR(255),
        rainfall REAL,
        temperature REAL
        )
        ;""")
        conn.commit()


        #matches_teams
        cursor.execute("""
        CREATE TABLE matches_teams(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        match_id INTEGER,
        team_id INTEGER,
        home INTEGER,
        team_goal INTEGER,
        points INTEGER,
        FOREIGN KEY (match_id)
        REFERENCES matches(id),
        FOREIGN KEY (team_id)
        REFERENCES teams(id)
        )
        ;""")
        conn.commit()

        #players
        cursor.execute("""
        CREATE TABLE players(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name VARCHAR(255),
        team_id INTEGER,
        position VARCHAR(255),
        birthday DATE,
        nationality VARCHAR(255),
        FOREIGN KEY (team_id)
        REFERENCES teams(id)
        )
        ;""")
        conn.commit()

        #goals
        cursor.execute("""
        CREATE TABLE goals(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        player_id INTEGER,
        match_id INTEGER,
        goal_type VARCHAR(255),
        FOREIGN KEY (match_id)
        REFERENCES matches(id),
        FOREIGN KEY (player_id)
        REFERENCES players(id)
        )
        ;""")
        conn.commit()


In [63]:
db = ChampionshipsDataBase()

<IPython.core.display.Javascript object>

In [64]:
db.get_tables_names()

['championships',
 'sqlite_sequence',
 'teams',
 'participation',
 'matches',
 'matches_teams',
 'players',
 'goals']

In [65]:
db.get_df('sqlite_sequence')

,name,seq


#### create championship table sql

In [66]:
#1.championships
db.get_df('championships')
db.cur.execute("""
        INSERT INTO championships(
        name,
        country,
        start_year,
        end_year) VALUES ('Ligue 1', 'France', '2020', '2021');""")

In [67]:
db.get_df('championships')

,id,name,country,start_year,end_year
0,1,Ligue 1,France,2020,2021


#### create teams table in sql

In [70]:
#2.teams

display(db.get_df('teams'))
df_teams.head()

,id,name,city,coach_name,year_founded


,city,name,coach_name,president,year_founded,championship
0,Angers,Angers Sporting Club de l'Ouest,Gérald BATICLE,Saïd Chabane,1919,Ligue 1
1,Bordeaux,Football Club des Girondins de Bordeaux,Jean-Louis GASSET,Frédéric Longuépée,1881,Ligue 1
2,Brest,Stade Brestois 29,Michel DER ZAKARIAN,Denis Le Saint,1950,Ligue 1
3,Dijon,Dijon Football Côte-d'Or,David LINARES,Olivier Delcourt,1998,Ligue 1
4,Lens,Racing Club de Lens,Franck HAISE,Joseph Oughourlian,1906,Ligue 1


In [73]:

# insert new row into the table participation preventing dublicates
for i in range(len(df_teams)):
    name = df_teams.loc[i, 'name']
    city = df_teams.loc[i, 'city']
    coach_name = df_teams.loc[i, 'coach_name']
    year_founded = df_teams.loc[i, 'year_founded']
    
    db.cur.execute("""INSERT INTO teams(name, city, coach_name, year_founded) 
SELECT "{}", "{}", "{}", "{}" 
WHERE NOT EXISTS(SELECT * FROM teams WHERE name = "{}" AND city = "{}" AND coach_name = "{}" 
AND year_founded = "{}");"""
                   .format(name, city, coach_name, year_founded, name, city, coach_name, year_founded))       

In [72]:
db.get_df('teams').tail(3)

,id,name,city,coach_name,year_founded
17,18,Stade Rennais Football Club,Rennes,Bruno GENESIO,1901
18,19,Association Sportive de St-Etienne,Saint-Étienne,Claude PUEL,1919
19,20,Racing Club de Strasbourg Alsace,Strasbourg,Julien STEPHAN,1906


#### create participation table  

In [75]:
# insert new row into the table participation preventing dublicates
def update_participation_table(df_teams):
    for i in range(len(df_teams)):
        championship_id = db.cur.execute(
            "SELECT id FROM championships WHERE name = '{}';".format(
                df_teams.loc[i, 'championship'])).fetchone()[0]
        team_id = db.cur.execute("SELECT id FROM teams WHERE city = '{}';".format(
            df_teams.loc[i, 'city'])).fetchone()[0]
        db.cur.execute("""INSERT INTO participation(championship_id, team_id) 
    SELECT '{}', '{}' 
    WHERE NOT EXISTS(SELECT * FROM participation WHERE championship_id = '{}' AND team_id = '{}');"""
                       .format(championship_id, team_id, championship_id, team_id))
        
update_participation_table(df_teams)

In [76]:
db.get_df('participation').tail(3)

,id,championship_id,team_id
17,18,1,18
18,19,1,19
19,20,1,20


#### create players table 

In [77]:
#4 matches team
db.get_df('players')

,id,name,team_id,position,birthday,nationality


In [78]:
df_players.head()

,number,name,country,birthday,position,club
0,1.0,P. Bernardoni,FRA,1997-04-18,Gar.,Angers
1,16.0,L. Butelle,FRA,1983-04-3,Gar.,Angers
2,30.0,D. Petkovic,MNE,1993-05-25,Gar.,Angers
3,25.0,A. Bamba,CIV,1990-04-25,Déf.,Angers
4,20.0,K. Boma,FRA,2002-11-20,Déf.,Angers


In [79]:
# insert new row into the table participation preventing dublicates
for i in range(len(df_players)):
    name = df_players.loc[i, 'name']
    team_id = db.cur.execute("SELECT id FROM teams WHERE city = '{}';".format(
        df_players.loc[i, 'club'])).fetchone()[0]
    position = df_players.loc[i, 'position']
    birthday = df_players.loc[i, 'birthday']
    nationality = df_players.loc[i, 'country']
    #print(name, team_id, position, birthday, nationality)
    
    db.cur.execute("""INSERT INTO players(name, team_id, position, birthday, nationality) 
SELECT "{}", "{}", "{}", "{}", "{}" 
WHERE NOT EXISTS(SELECT * FROM players WHERE name = "{}" AND team_id = "{}" AND position = "{}" 
AND birthday = "{}" AND nationality = "{}");"""
                   .format(name, team_id, position, birthday, nationality, name, team_id, position, birthday, nationality))

In [81]:
db.get_df('players').head(4)

,id,name,team_id,position,birthday,nationality
0,1,P. Bernardoni,1,Gar.,1997-04-18,FRA
1,2,L. Butelle,1,Gar.,1983-04-3,FRA
2,3,D. Petkovic,1,Gar.,1993-05-25,MNE
3,4,A. Bamba,1,Déf.,1990-04-25,CIV


In [84]:
#db.cur.execute("SELECT name, birthday  FROM players WHERE YEAR(birthday) > '2000';")


#### create matches table 

In [89]:
#5 players
db.get_df('matches')

,id,date,place,rainfall,temperature


In [86]:
df_matches.tail()

,date,place,temperature,rainfall
375,2021-05-23T21:00:00+0200,Rennes,18,0
376,2021-05-23T21:00:00+0200,Nantes,18,0
377,2021-05-23T21:00:00+0200,Saint-Étienne,18,0
378,2021-05-23T21:00:00+0200,Reims,18,0
379,2021-05-23T21:00:00+0200,Strasbourg,18,0


In [99]:
# insert new row into the table participation preventing dublicates
for i in range(len(df_matches)):
    date = df_matches.loc[i, 'date']
    place = df_matches.loc[i, 'place']
    temperature = df_matches.loc[i, 'temperature']
    rainfall = df_matches.loc[i, 'rainfall']
    
    db.cur.execute("""INSERT INTO matches(date, place, temperature, rainfall) 
SELECT "{}", "{}", "{}", "{}"
WHERE NOT EXISTS(SELECT * FROM matches WHERE date = "{}" AND place = "{}" AND temperature = "{}" 
AND rainfall = "{}");"""
                   .format(date, place, temperature, rainfall, date, place, temperature, rainfall))

In [100]:
db.get_df('matches')

,id,date,place,rainfall,temperature
0,1,2020-08-21T19:00:00+0200,Bordeaux,0.0,18.0
1,2,2020-08-22T17:00:00+0200,Dijon,0.0,18.0
2,3,2020-08-22T21:00:00+0200,Lille,0.0,18.0
3,4,2020-08-23T13:00:00+0200,Monaco,0.0,18.0
4,5,2020-08-23T15:00:00+0200,Lorient,0.0,18.0
...,...,...,...,...,...
375,376,2021-05-23T21:00:00+0200,Rennes,0.0,18.0
376,377,2021-05-23T21:00:00+0200,Nantes,0.0,18.0
377,378,2021-05-23T21:00:00+0200,Saint-Étienne,0.0,18.0
378,379,2021-05-23T21:00:00+0200,Reims,0.0,18.0


#### create matches_teams table 

In [144]:
db.get_df('matches_teams')

,id,match_id,team_id,home,team_goal,points


In [98]:
df_matches_teams.head()

,date,place,team,team_goal,players
0,2020-08-21T19:00:00+0200,Bordeaux,Bordeaux,0,[]
1,2020-08-21T19:00:00+0200,Bordeaux,Nantes,0,[]
2,2020-08-22T17:00:00+0200,Dijon,Dijon,0,[]
3,2020-08-22T17:00:00+0200,Dijon,Angers,1,[I. Traoré]
4,2020-08-22T21:00:00+0200,Lille,Rennes,1,[D. Da Silva]


In [104]:
# insert new row into the table participation preventing dublicates
for i in range(len(df_matches_teams)):
    match_id = db.cur.execute("SELECT id FROM matches WHERE date = '{}' AND place = '{}';".format(
        df_matches_teams.loc[i, 'date'], df_matches_teams.loc[i, 'place'])).fetchone()[0]
    team_id = db.cur.execute("SELECT id FROM teams WHERE city = '{}';".format(
        df_matches_teams.loc[i, 'team'])).fetchone()[0]
    home = df_matches_teams.loc[i, 'team'] == df_matches_teams.loc[i, 'place']
    team_goal = df_matches_teams.loc[i, 'team_goal']
    points = 99
    #print(name, team_id, position, birthday, nationality)
    
    db.cur.execute("""INSERT INTO matches_teams(match_id, team_id, home, team_goal, points) 
SELECT "{}", "{}", "{}", "{}", "{}" 
WHERE NOT EXISTS(SELECT * FROM matches_teams WHERE match_id = "{}" AND team_id = "{}" AND home = "{}" 
AND team_goal = "{}" AND points = "{}");"""
                   .format(match_id, team_id, home, team_goal, points, match_id, team_id, home, team_goal, points))

In [105]:
db.get_df('matches_teams').tail()

,id,match_id,team_id,home,team_goal,points
755,756,373,5,True,0,99
756,757,372,3,True,0,99
757,758,371,1,True,1,99
758,759,379,2,False,2,99
759,760,380,7,False,1,99


#### create goals table 

In [146]:
db.get_df('goals')

,id,player_id,match_id,goal_type


In [124]:
# insert new row into the table participation preventing dublicates
count = 0
players_not_in_list_names = []
players_not_in_list_clubs = []
for i in range(len(df_matches_teams)):
    club = df_matches_teams.loc[i, 'team']
    for player_name in df_matches_teams.loc[i, 'players']:
        #print(player_name)
        
        try:
            player_id = db.cur.execute("""SELECT id FROM players WHERE name = "{}";""".format(player_name)).fetchone()[0]
            #print(player_name, player_id)
        except:
            players_not_in_list_names.append(player_name)
            players_not_in_list_clubs.append(club)
            count +=1
print(players_not_in_list_clubs)

# insert new row into the table participation preventing dublicates
for (name, club) in zip(players_not_in_list_names, players_not_in_list_clubs):
    team_id = db.cur.execute("SELECT id FROM teams WHERE city = '{}';".format(club)).fetchone()[0]
    position = None
    birthday = None
    nationality = None
    #print(name, team_id, position, birthday, nationality)
    
    db.cur.execute("""INSERT INTO players(name, team_id, position, birthday, nationality) 
SELECT "{}", "{}", "{}", "{}", "{}" 
WHERE NOT EXISTS(SELECT * FROM players WHERE name = "{}" AND team_id = "{}" AND position = "{}" 
AND birthday = "{}" AND nationality = "{}");"""
                   .format(name, team_id, position, birthday, nationality, name, team_id, position, birthday, nationality))
    


['Nîmes', 'Dijon', 'Bordeaux', 'Marseille', 'Rennes', 'Bordeaux', 'Marseille', 'Strasbourg', 'Marseille', 'Strasbourg', 'Lyon', 'Rennes', 'Bordeaux', 'Strasbourg', 'Marseille', 'Marseille', 'Lens', 'Bordeaux']


In [125]:
db.get_df('players')


,id,name,team_id,position,birthday,nationality
0,1,P. Bernardoni,1,Gar.,1997-04-18,FRA
1,2,L. Butelle,1,Gar.,1983-04-3,FRA
2,3,D. Petkovic,1,Gar.,1993-05-25,MNE
3,4,A. Bamba,1,Déf.,1990-04-25,CIV
4,5,K. Boma,1,Déf.,2002-11-20,FRA
...,...,...,...,...,...,...
636,637,M. Niang,18,None,None,None
637,638,Pablo,2,None,None,None
638,639,N. Radonjic,9,None,None,None
639,640,D. Pereira Da Costa,5,None,None,None


In [127]:
for i in range(len(df_matches_teams)):
    club = df_matches_teams.loc[i, 'team']
    for player_name in df_matches_teams.loc[i, 'players']:
        player_id = db.cur.execute("""SELECT id FROM players WHERE name = "{}";""".format(player_name)).fetchone()[0]

        match_id = db.cur.execute("SELECT id FROM matches WHERE date = '{}' AND place = '{}';".format(
            df_matches_teams.loc[i, 'date'], df_matches_teams.loc[i, 'place'])).fetchone()[0]
        goal_type = 'Normal' 

        db.cur.execute("""INSERT INTO goals(player_id, match_id, goal_type) 
    SELECT "{}", "{}", "{}" 
    WHERE NOT EXISTS(SELECT * FROM goals WHERE player_id = "{}" AND match_id = "{}" AND goal_type = "{}" );"""
                       .format(player_id, match_id, goal_type, player_id, match_id, goal_type))

In [128]:
db.get_df('goals')

,id,player_id,match_id,goal_type
0,1,12,2,Normal
1,2,534,3,Normal
2,3,172,3,Normal
3,4,310,4,Normal
4,5,308,4,Normal
...,...,...,...,...
949,950,244,374,Normal
950,951,21,371,Normal
951,952,49,379,Normal
952,953,45,379,Normal
